In [1]:
# Preliminaries
import json
import requests
import subprocess
import time
import boto3
from IPython.display import display, HTML

stack_name = 'bace-app' # Replace with the stack name created with `sam deploy --guided`, which is saved in the SAM configuration file (samconfig.toml by default).
output_key = 'BaceApi' # See under Outputs in `template.yaml`
# Update `region` below to the region created with `sam deploy --guided`, saved in the SAM configuration file (samconfig.toml by default)
#   if different from the default region in ~/.aws/config (or C:\Users\USERNAME\.aws\config)
region = boto3.Session().region_name # example if different from default: region = 'us-east-2'
# os.environ['AWS_PROFILE'] = "YOUR_AWS_PROFILE_NAME" # Set this if your current AWS login profile is not the default one -- see profiles in ~/.aws/config (or C:\Users\USERNAME\.aws\config)

# Use subprocess to run the AWS CLI command
command = f'aws cloudformation describe-stacks --stack-name {stack_name} --region {region} --query "Stacks[].Outputs[?OutputKey==\'{output_key}\'].OutputValue" --output text'
result = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

# Extract the output from the command result
base_url = result.stdout.decode('utf-8').strip()
print(f'URL for your API\n{base_url}')

URL for your API
https://r3ue9e2bf3.execute-api.us-east-2.amazonaws.com/Prod/


# Homepage

GET Request to test that the application is up and running.

We recommend running this block each time you push a new version of BACE so that the machine gets warmed up.
Otherwise the initial call may take 10 to 20 seconds for warming up.

In [2]:
print('Initial call may take a moment due to cold state of machine...\n')

# First Call
t0 = time.time()
route = "/"
homepage = requests.get(
    f'{base_url}{route}'
)
print('Response:')
print(json.loads(homepage.text))
print(f'Time to receive first call: {time.time() - t0}s\n')

print('Second call should be shorter if machine was dormant...\n')
# Second Call
t0 = time.time()
route = "/"
homepage = requests.get(
    f'{base_url}{route}'
)
print(f'Time to receive second call: {time.time() - t0}s')

Initial call may take a moment due to cold state of machine...

Response:
{'message': 'Hello! Your BACE application is up and running.', 'author': 'Pen Example Application'}
Time to receive first call: 24.2078378200531s

Second call should be shorter if machine was dormant...

Time to receive second call: 0.15543913841247559s


# Random Design

GET request to receive a random design

In [3]:
route = "/random_design"
design = requests.get(
    f'{base_url}{route}'
)

print(json.loads(design.text))

{'color_a': 'Black', 'color_b': 'Blue', 'price_a': 1.1720134431400515, 'price_b': 4.447465373096071, 'type_a': 'Gel', 'type_b': 'Gel'}


# Route: /create_profile

API request to create a new profile for an individual.

In [4]:
route = "/create_profile"

data = {
    "survey_id": "test_colab",
    # Include additional variables that you want stored with an individual's profile here.
}

r = requests.post(
    f'{base_url}{route}',
    data=data
)

r = json.loads(r.text)
profile_id = r.get('profile_id')

print(r)

try:
    display(HTML(r['message_0_1']))
    display(HTML(r['message_1_1']))
except:
    print(r)

{'profile_id': 'e78ed168-6223-4e93-a0a0-ce6c1870cb53', 'color_a_1': 'Blue', 'color_b_1': 'Black', 'price_a_1': 3.051616111643358, 'price_b_1': 1.8828288121214483, 'type_a_1': 'Gel', 'type_b_1': 'Ballpoint', 'message_0_1': '\n        <table width=\'300px\' border=\'1\' cellpadding=\'1\' cellspacing=\'1\' style=\'font-family: Arial, Tahoma, "Helvetica Neue", Helvetica, sans-serif; border-collapse:collapse; background-color:#eee9e7; color:black;\'>\n            <tbody>\n                <tr>\n                    <th style="text-align: center; background-color: #ded4ce;"><b>Pen A</b></th>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Price:</em><br> $3.05</td>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Pen Color:</em><br> Blue</td>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Pen Type:</em><br> Gel</td>\n                <

Pen A
Price: $3.05
Pen Color: Blue
Pen Type: Gel


Pen B
Price: $1.88
Pen Color: Black
Pen Type: Ballpoint


# Route /update_profile

API request to update profile for an individual

In [5]:
##### Answer to previous question ########
answer = 1
##########################################

route = "/update_profile"
data = {
    "profile_id": profile_id,
    "answer": answer
}

r = requests.post(
    f'{base_url}{route}',
    data=data
)

r = json.loads(r.text)

question_number = list(r.keys())[0].rpartition('_')[-1]

try:
    display(HTML(r[f'message_0_{question_number}']))
    display(HTML(r[f'message_1_{question_number}']))
except:
    print(r)

Pen A
Price: $0.89
Pen Color: Blue
Pen Type: Ballpoint


Pen B
Price: $1.35
Pen Color: Black
Pen Type: Gel


# Posterior Estimates

Route: /estimates

## POST
API request to update profile with a given answer and return estimates

In [6]:
##### Answer to previous question ########
answer = 1
##########################################

route = "/estimates"
data = {
    "profile_id": profile_id,
    "answer": answer
}

r = requests.post(
    f'{base_url}{route}',
    data=data
)

r = json.loads(r.text)
print(r)

{'blue_ink': {'mean': -1.0807718211387642, 'median': -1.1588708512663266, 'std': 0.6206758708352549}, 'gel_pen': {'mean': 0.9170514174472643, 'median': 0.9081326760738433, 'std': 0.7465664841923879}, 'mu': {'mean': 5.570841133681229, 'median': 5.596005996831457, 'std': 2.590838335460322}}


## GET
API request to return estimates for a given profile

In [7]:
route = "/estimates"
params = {
    "profile_id": profile_id,
}

r = requests.get(
    f'{base_url}{route}',
    params=params
)

r = json.loads(r.text)

print(r)

{'mu': {'median': '5.596005996831457', 'mean': '5.570841133681229', 'std': '2.590838335460322'}, 'blue_ink': {'median': '-1.1588708512663266', 'mean': '-1.0807718211387642', 'std': '0.6206758708352549'}, 'gel_pen': {'median': '0.9081326760738433', 'mean': '0.9170514174472643', 'std': '0.7465664841923879'}}
